# Preamble

In [27]:
import numpy as np
import itertools as it
import time

# Class definitions

In [28]:
class game:

    def __init__(self, N, S, U):
        self.N = N
        self.S = S
        self.U = U
        self.n = len(N)
    
    def best_response(self,i,s):
        
        # Modify s with all actions from i
        profs = [ s[:i] + (x,) + s[(i+1):] for x in self.S[i] ]
            
        # Find utilities
        utils = [self.U(x,i) for x in profs]
        
        # Find utility maximizing actions
        best = [self.S[i][x] for x in range(0,len(utils)) if utils[x] == max(utils)]
        
        return best
    
    def joint_best_response(self,s):
        
        br = [self.best_response(x,s) for x in range(self.n)]
        
        return br
    
    def joint_util(self,s):
        
        util = [self.U(s,x) for x in range(self.n)]
        return(util)
        
    def eval_util(self,i,si,s):
        """
        This function evaluates the utilities of all actions in si for
        individual i, given that the rest of agents act according to s
        """
        
        if len(si) == 0:
            
            return([-float("Inf")])
            
        else:
            
            # Generate strategy profiles
            profs = [ s[:i] + (x,) + s[(i+1):] for x in si ]
            
            utils = [self.U(x,i) for x in profs]
            
            return(utils)
            
    def nash_eq_exhaus(self):
        
        # Generate strategy space
        S = list(it.product(*self.S))
        
        # Find nash equiibria
        neq = [s for s in S if all([s[x] in self.joint_best_response(s)[x] for x in range(self.n)])]
        
        return(set(neq))
        
    
class sm_game(game):
    
    def sup_eq(self):
        
        a0 = tuple([max(x) for x in self.S])
        a1 = tuple([max(x) for x in self.joint_best_response(a0)])
        
        while a0 != a1:
            
            a0 = a1[:]
            a1 = tuple([max(x) for x in self.joint_best_response(a0)])
            
        return(a1)
    
    def restrict(self,s):
        
        # Reduce the strategy space to those greater or equal to s
        restr_s = [ [y for y in self.S[x] if y >= s[x] ] for x in range(self.n) ]
        
        return( sm_game(self.N, restr_s, self.U) )
    
    
    def inf_eq(self):
        
        # Find least action of all players
        a0 = tuple([min(x) for x in self.S])
        a1 = tuple([min(x) for x in self.joint_best_response(a0)])
        
        while a0 != a1:
            
            a0 = a1[:]
            a1 = tuple([min(x) for x in self.joint_best_response(a0)])
            
        return(a1)
    
    def echenique(self):
        
        # Find extremal equilibria
        infeq = self.inf_eq()
        supeq = self.sup_eq()
        
        if infeq == supeq:
            return set([infeq])
        
        # Initialize equilibrium list
        eq_list = set([infeq,supeq])
        
        # Initialize state
        M = set([(infeq,infeq)])
        
        # Find stopping state
        M_final = set([(supeq,
                        tuple([min(x) for x in self.joint_best_response(supeq)]))])
    
        while M != M_final:
            
            M_prime = set()
            
            for s_s in M:
                
                s = s_s[0][:]
                s_star = s_s[1][:]
                
                for i in range(self.n):
                    
                    if s[i] < supeq[i]:
                    
                        s_greater = list(s)
                        s_greater[i] = s_greater[i] + 1
                        s_greater = tuple(s_greater)
                        
                        # Create a restricted game
                        gamma_r = self.restrict(s_greater)
                        
                        # S hat
                        s_hat = gamma_r.inf_eq()
                        
                        # Find utilities of profile s_Hat
                        u_hat = self.joint_util(s_hat)
                        
                        # Find a restricted set of strategies
                        z = [ [y for y in self.S[x] if s_star[x] <= y and y < s_greater[x]] for x in range(self.n) ]
                        
                        # Find maximum utilities that all agents would receive from
                        # deviating from s_hat to a strategy in z[i]
                        u_alt = [ max(self.eval_util(x,z[x],s_hat)) for x in range(self.n)]
                        
                        # Check if any player is strictly better off switching.
                        # If none is, s_hat is an equilibrium
                        if not any([ u_alt[x] > u_hat[x] for x in range(self.n)]):
                            
                            eq_list.add(s_hat)
                        
                        # Find inf best response (s_hat)
                        brep = tuple([min(x) for x in self.joint_best_response(s_hat)])
                        
                        # Add to state
                        M_prime.add((s_hat,brep))

            M = M_prime.copy()
            #print("M=",[x[0] for x in M])
            
        return(eq_list)

# Section 5 Example

In [58]:
# Create game
N = [1, 2]
S = [[1, 2, 3, 4], [1, 2, 3, 4]]
def util(s, i):

    if i == 0:
        
        pay = np.matrix([[3,3,3,0],
                         [2,2,2,0],
                         [1,1,1,0],
                         [0,0,0,0]])
        
    elif i == 1:
        
        pay = np.matrix([[3,2,1,0],
                         [3,2,1,0],
                         [3,2,1,0],
                         [0,0,0,0]])
    
    return pay[s[0]-1,s[1]-1]

game = sm_game(N, S, util)

t0 = time.time()
eq = game.nash_eq_exhaus()
t1 = time.time()
print("Exhaustive result:",eq)
print("Exhaustive time:",t1-t0)

t0 = time.time()
eq = game.echenique()
t1 = time.time()
print("Echenique (2007) result:",eq)
print("Echenique (2007) time:",t1-t0)

Exhaustive result: {(4, 4), (1, 1)}
Exhaustive time: 0.0
Echenique (2007) result: {(4, 4), (1, 1)}
Echenique (2007) time: 0.0


# Section 7 game

In [60]:
# Parameters
K = 50
alpha = (0.5,0.5)
beta = (0.5,0.5)

# create game
N = [1,2]
S = [range(0,K+1) for x in range(2)]
def util(s, i, K, alpha, beta):
    s = tuple([100*x/K for x in s])
    u = -0.01*alpha[i]*(s[i]-s[1-i])**2 + 2*beta[i]*np.sin(100*s[i]) + 0.0001*( (1-alpha[i])*s[i]*(1+s[1-i]) - 0.01*(0.5-beta[i])*s[i]**2 )
    return(u)

u = lambda s,i: util(s,i,K,alpha,beta)

game2 = sm_game(N,S,u)

t0 = time.time()
eq = game2.nash_eq_exhaus()
t1 = time.time()
print("Exhaustive result:",eq)
print("Exhaustive time:",t1-t0)

t0 = time.time()
eq = game2.echenique()
t1 = time.time()
print("Echenique (2007) result:",eq)
print("Echenique (2007) time:",t1-t0)

Exhaustive result: {(16, 16), (22, 22), (10, 10), (46, 46), (28, 28), (4, 4), (40, 40), (34, 34)}
Exhaustive time: 2.422001600265503
Echenique (2007) result: {(16, 16), (22, 22), (10, 10), (4, 4), (28, 28), (34, 34), (40, 40), (46, 46)}
Echenique (2007) time: 0.18752050399780273
